In [ ]:
import numpy as np
import sys
import random
import copy
import string
import time
import datetime
from tqdm import tqdm
from scipy.stats import pareto
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
rng = np.random.default_rng(int(time.time()))

In [ ]:
##########################################
#### Functions for plotting Functions  ###
##########################################
exec(open('../matplot-lib-utils.py').read())

In [ ]:
##################################################
####### Fetch: General Helper Functions  #########
##################################################
exec(open('../utils.py').read())

In [ ]:
SAME_G = True

DEBUG = False
def print_debug(s):
    if DEBUG: print(s)


In [ ]:
##########################################
### Fetch: Algorithms and Baseline ######
##########################################
exec(open('../algorithms.py').read())

In [ ]:
def generator(beta, n, m, T, p, alpha, f, delta):
    n_f = int(n*f)

    lat_util = np.zeros((n,3))

    # Initialize utilities
    rand_bits = (rng.random(n) < p)
    
    items = pareto.rvs(delta, size=n) # other artists
    for i in range(n):
        jj = int(rng.random()*m)
        lat_util[i][jj] = items[i]
    
    # generate protected groups
    grp_men = set(rng.choice(n, n - n_f, replace=False))
    all = set([i for i in range(n)])
    grps = [grp_men, all.difference(grp_men)]

    # generate observed utility
    obs_util = copy.deepcopy(lat_util)
    for i in grps[1]:
        for j in range(m):
            obs_util[i][j] *= beta

    print_debug((obs_util == lat_util).all())
    
    return lat_util, obs_util, grps


def get_details_top_k(sol, k):
    other_artists = set([i+n_alpha for i in range(n-n_alpha)])
    other_artists = other_artists.intersection(sol)
    other_artists_utils  = marg_F_mult(set(), other_artists, lat_util, m)
    top_5_other_artists = get_top_n(other_artists, other_artists_utils, k)
    print("other: ")
    for i in top_5_other_artists:
        print_details(i, lat_util, obs_util)

    emerging_artists = set([i for i in range(n_alpha)])
    emerging_artists = emerging_artists.intersection(sol)
    emerging_artists_utils  = marg_F_mult(set(), emerging_artists, lat_util, m)
    top_5_emerging_artists = get_top_n(emerging_artists, emerging_artists_utils, k)
    print('')
    print("emerg: ")
    for i in top_5_emerging_artists:
        print_details(i, lat_util, obs_util)

In [ ]:
# parameters
ITER = -1; n = 0; k = 0; m = 0
T = 0; p = 0; alpha = -1
f_list = []
delta_list = []

f = -1; delta = -1

SAME_G = True
func = lambda x: np.log(1 + x) 
func2 = lambda x: np.log(1 + x)

def run(T=1, p=0.9, alpha=0.0, f=0.5, delta=1):
    SAME_G = True
    func = lambda x: np.log(1 + x) 
    func2 = lambda x: np.log(1 + x)
    
    # T: max plays of emerging artist
    # p: probability new song (seems robust to the choice of p)
    # alpha: fraction of emerging artists

    # parameters
    ITER = 50
    n = 250
    k = 50
    m = 3

    DEBUG = False

    x=[]
    y_uncons=[]
    y_iid=[]
    y_disj=[]
    y_uncons_err=[]
    y_iid_err=[]
    y_disj_err=[]


    for beta in [0.01, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95, 1]:
        list_opt = []
        list_baseline_uncons = []
        list_alg_iid = []
        list_algo_disj = []

        for ijk in range(ITER):
            lat_util, obs_util, grps = generator(beta, n, m, T, p, alpha, f, delta)
            n_alpha = int(n*alpha)
            n_f = int(n*f)

            # run algorithms and baselines
            sol_opt = baseline_uncons(lat_util, grps, k, m)
            sol_baseline_uncons = baseline_uncons(obs_util, grps, k, m)
            sol_alg_iid = algo_iid(obs_util, grps, k, m)
            sol_algo_disj = algo_disj(obs_util, grps, k, m)

            util_opt = F(sol_opt, lat_util, m)
            util_baseline_uncons = F(sol_baseline_uncons, lat_util, m)
            util_alg_iid = F(sol_alg_iid, lat_util, m)
            util_algo_disj = F(sol_algo_disj, lat_util, m)

            list_baseline_uncons.append(util_baseline_uncons / util_opt)
            list_alg_iid.append(util_alg_iid / util_opt)
            list_algo_disj.append(util_algo_disj / util_opt)

            emerging_artists = set([i for i in range(n_alpha)])
            other_artists = set([i+n_alpha for i in range(n-n_alpha)])

            p_grp = [[] for j in range(m)]
            for i in range(n):
                for j in range(m):
                    if lat_util[i][j] > 0: p_grp[j].append(i)

            for j in range(m): p_grp[j] = set(p_grp[j])

            sol_opt = set(sol_opt)
            sol_baseline_uncons = set(sol_baseline_uncons)
            sol_alg_iid = set(sol_alg_iid)
            sol_algo_disj = set(sol_algo_disj)

        y_uncons.append( np.mean(list_baseline_uncons) )
        y_iid.append( np.mean(list_alg_iid) )
        y_disj.append( np.mean(list_algo_disj) )
        y_uncons_err.append( np.std(list_baseline_uncons) / np.sqrt(ITER) )
        y_iid_err.append( np.std(list_alg_iid) / np.sqrt(ITER) )
        y_disj_err.append( np.std(list_algo_disj) / np.sqrt(ITER) )

        x.append(beta)

    fig, ax = plt.subplots()
    x = np.array(x)
    plt.errorbar(x, y_uncons, yerr=y_uncons_err, color="red", label='Uncons',  linewidth=8, alpha=1.0)
    plt.errorbar(x, y_iid, yerr=y_iid_err, linestyle='-', color="#F2B93F", label='Algorithm 1 (IID)',  linewidth=8, alpha=1.0)
    plt.errorbar(x, y_disj, yerr=y_disj_err, linestyle='-.', color="#604EE6", label='Algorithm 2 (Disjoint)',  linewidth=8, alpha=1.0)



    plt.title(f'Simulation with parameters:'
             +f'n={n}, k={k}, frac.-of-non-men={f},SAME_G={SAME_G} (log(1+x))\n'
             +f'm={m}, power-law-dist (delta={delta}), ITER={ITER}, weight_F={np.round(weight_F,2)}\n\n', fontsize=18)
    plt.ylim(0.75, 1.01)
    ax.set_ylabel('Normalized Latent Utility',fontsize=32)
    ax.set_xlabel('$\\beta$', fontsize=34)
    legend = plt.legend(loc='best', shadow=False, fontsize=26)
    plt.tick_params(axis='both', which='major', labelsize=26)
    
    
    print()
    print(f'y_uncons: {y_uncons}')
    print(f'y_iid: {y_iid}')
    print(f'y_disj: {y_disj}')
    print(f'y_uncons_err: {y_uncons_err}')
    print(f'y_iid_err: {y_iid_err}')
    print(f'y_disj_err: {y_disj_err}')
    
    
    # plt.show()
    pdf_savefig()


In [ ]:

f_list = [0.25, 0.5, 0.75]
delta_list = [1.0, 1.2, 1.5, 2, 2.5, 3.0]
weight_F = [1, 1, 1]

for f in f_list:
    for delta in tqdm(delta_list):
        run(T=1, p=0.9, alpha=0.0, f=f, delta=delta)